# Data Wrangling Part 5 - Pandas Part 2

The following report was submitted as part of my Master of Data Science coursework on Data Wrangling. This report is the final in a five part series where I showcase work on various topics relating to Data Wrangling. This report is the second of two reports on the popular library Pandas. 

The main scope of this report was to demonstrate how Pandas can emulate SQL queries and provide similar outputs. An exploration of the performance of Pandas and SQL was done to provide insight into whether an SQL database or a pandas is a better option for your data analysis project.

## Working with *pandas* Data Frames (Part 2)

&emsp; **Author:** Daniel Gladman <br>
&emsp; **Originally Written:** 2023-04-13 <br>

This report will use each of datasets belonging to the well-known NYCFlights13 package, which includes datasets consisting of flights, weather, planes, airlines and airports. The data was downloaded and saved prior to beginning the tasks in this report. The purpose of this report is to compare data querying methods, syntaxes and performance between native pandas and SQL.

### Libraries

First the libraries used in this task shall be imported.

In [1]:
import sqlite3
import pandas as pd
import os

SQLite is a software library that is included within base python and provides a serverless relational database management system (RDBMS). SQLite is able to store and manage data in a single .db file.

Pandas is a powerful external library that is built ontop of Numpy and is used primarily for data manipulation, analysis, and cleaning. OS is an inbuilt python library that facilitates interaction with the operating system via python.

Before any experiment, it is ideal to declare a prediction (or hypothesis) before performing any analysis, and even more ideally it is better for the hypothesis to be grounded in some theory. Given that in order to return an output using SQL queries, the data must be extracted from the database and maniuplated into a pandas DataFrame, it is expected that the native pandas queries will be quicker than the SQL queries. When querying directly on a pandas dataframe using pandas, the lack of extraction should naturally make the process faster.

### Load the data.

First, the datasets will be loaded directly into pandas and then into SQLite

In [2]:
datasets = ['airlines', 'airports', 'flights', 'planes', 'weather']
db_path = './data/nycflights13.db'

airlines, airports, flights, planes, weather = [pd.read_csv(f"./data/nycflights13_{dataset}.csv.gz", comment="#") for dataset in datasets]
conn = sqlite3.connect(db_path)

if not os.path.exists(db_path):
    airlines.to_sql("airlines", conn)
    airports.to_sql("airports", conn)
    flights.to_sql("flights", conn)
    planes.to_sql("planes", conn)
    weather.to_sql("weather", conn)

## Task 1: "SELECT DISTINCT" on Single Column

The first SQL query returns the unique values within a specific column. In this case, it will be the unique values within the engine column in the planes database.

In [3]:
task1_sql=pd.read_sql_query("""SELECT DISTINCT engine FROM planes""", conn)
task1_sql

,engine
0,Turbo-fan
1,Turbo-jet
2,Reciprocating
3,4 Cycle
4,Turbo-shaft
5,Turbo-prop


#### **PANDAS Equivalent of "SELECT DISTINCT"**

Two proposed solutions shall be executed.

In [4]:
task1_my = pd.DataFrame(planes['engine'].unique(), columns=['engine'])
try:
    pd.testing.assert_frame_equal(task1_sql, task1_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


The first solution involves creating a new dataframe using the output of the pandas.unique() method. The pandas.unique() method uses a hash table to return the unique values in a series. Uniques are returned in order of appearance according to the documentation. 

When this method is called on a the planes\['engine'\], it produces the desired information. However, to ensure that the format matches the output of the SQL query, the unique values need to be assigned to a new DataFrame.

There is another method to achieve the same result but without creating a new DataFrame.

In [5]:
task1_my2 = planes[['engine']].drop_duplicates().reset_index(drop=True)
try:
    pd.testing.assert_frame_equal(task1_sql, task1_my2)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


In this code, the pandas.drop_duplicates() method is called on the pandas series to produce the same output. Dropping duplicate entries will result in only the unique entires remaining in the series. The difference in this case is that drop_duplicates() can be applied directly to a DataFrame object, whereas the pandas.unique() method can only be used on a series which must be converted back into a DataFrame to match the SQL output.

### **Task 1 - Performance test**

The results of the performance test are below.

In [6]:
%%timeit
pd.read_sql_query("""SELECT DISTINCT engine FROM planes""", conn)

651 µs ± 4.77 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%%timeit
pd.DataFrame(planes['engine'].unique(), columns=['engine'])

177 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
%%timeit
planes[['engine']].drop_duplicates().reset_index(drop=True)

482 µs ± 6.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


As expected, the native pandas methods performed better than the SQL query, with the unique() method performing considerably better than the drop_duplicates method. Where possible, it is better to use the pandas.unique() method in these situations.

## Task 2: "SELECT DISTINCT" on multiple columns.

The next SQL query returns the unique values within two specific columns. In this case, it will be the unique values within the engine and type columns in the planes database.

In [9]:
task2_sql=pd.read_sql_query("""SELECT DISTINCT type, engine FROM planes""", conn)
task2_sql

,type,engine
0,Fixed wing multi engine,Turbo-fan
1,Fixed wing multi engine,Turbo-jet
2,Fixed wing single engine,Reciprocating
3,Fixed wing multi engine,Reciprocating
4,Fixed wing single engine,4 Cycle
5,Rotorcraft,Turbo-shaft
6,Fixed wing multi engine,Turbo-prop


### **PANDAS Equivalent of "SELECT DISTINCT" on multiple columns.**

In this case, it is not possible to utilize the pandas.unique() method as that method only works on a single series of data, whereas in this case the unique values must be unique combinations of two columns.

Fortunately, the drop_duplicaties method is capable of handling this task.

In [10]:
task2_my = planes[['type', 'engine']].drop_duplicates().reset_index(drop=True)
try:
    pd.testing.assert_frame_equal(task2_sql, task2_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


As earlier the drop_duplicates method can be applied to multiple columns at once, producing the same output as the SQL query.

### **Task 2 - Performance test**

The results of the performance test are below.

In [11]:
%%timeit
pd.read_sql_query("""SELECT DISTINCT type, engine FROM planes""", conn)

900 µs ± 8.33 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
%%timeit
planes[['type', 'engine']].drop_duplicates().reset_index(drop=True)

877 µs ± 8.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Interestingly, this time the drop_duplciates method was only marginally better than the SQL query. It seems that perhaps as the queries become more complicated, SQL may begin to outperform pandas.

## Task 3: "SELECT COUNT / GROUP BY" on single column.

This SQL query  reads the "planes" table and counts the number of rows by the type of engine. It then sorts the results by engine in alphabetical.

In [13]:
task3_sql=pd.read_sql_query("""SELECT COUNT(*), engine FROM planes GROUP BY engine""", conn)
task3_sql

,COUNT(*),engine
0,2,4 Cycle
1,28,Reciprocating
2,2750,Turbo-fan
3,535,Turbo-jet
4,2,Turbo-prop
5,5,Turbo-shaft


### **PANDAS Equivalent of  "SELECT COUNT / GROUP BY" on single column.**

To replicate the SQL output (out of order indices as well), it is possible to do this in a roundabout way. 

In [14]:
engines = planes.groupby(planes['engine'])[['engine']].value_counts().index
engine_counts = planes.groupby(planes['engine'])[['engine']].value_counts().values
task3_my = pd.DataFrame([engine_counts.T, engines]).T
task3_my.columns = ['COUNT(*)', 'engine']
task3_my['COUNT(*)'] = task3_my['COUNT(*)'].astype("int64")


try:
    pd.testing.assert_frame_equal(task3_sql, task3_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


First, the code is grouping the data in the "planes" DataFrame by the type of engine, and then counting the number of occurrences of each engine type. The indices and counts are assigned to respective variables using the pandas.value_counts() method along with the index or values, which are then assigned into a new pandas DataFrame. However, for this DataFrame assignment to match the SQL output, a series of matrix transposes need to be called. First the output of the engine_counts needs to be transposed such that the DataFrame can be created. Then the DataFrame itself is transposed so the counts and indices are presented as columns. The columns are then renamed to match the SQL output. Finally, the counts are converted to int64, which is a quirk that arises from the value_counts() assignment. If this is not performed, the assert test will fail.

However, this is a very convoluted way to replicate the SQL output and is prone to errors (particularly with the dtype re-assignment). So, a different method is proposed.

In [15]:
task3_sql.sort_values(by="COUNT(*)", ascending=False, inplace=True)
task3_sql.reset_index(drop=True, inplace=True)
task3_sql

task3_my2 = planes['engine'].value_counts().reset_index()
task3_my2.columns = ['engine', 'COUNT(*)']
task3_my2 = task3_my2[['COUNT(*)', 'engine']]

try:
    pd.testing.assert_frame_equal(task3_sql, task3_my2)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


For this solution to work and pass the assert test, the SQL output must be reordered by the descending order of the counts instead of the engine. This is achieved by calling the pandas.sort_values() method. However, calling this will cuse the indices to fall out of sequential order, which will break the assert test. So the indices must also be reset.

This pandas code performs exactly the same operations using value_counts(), but it does so without the need to explicitly call the group_by method or create a new DataFrame; it performs grouping by perfoming value_counts on just the engine column. By calling reset_index(), the output will revert back to a DataFrame object instead of a summary table. However, for this DataFrame to pass the assert test, the columns need to be renamed to match the SQL output column names and also the order of the columns must be switched.

### **Task 3 - Performance test**

The results of the performance test are below.

In [16]:
%%timeit
pd.read_sql_query("""SELECT COUNT(*), engine FROM planes GROUP BY engine""", conn)

1.3 ms ± 16.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
%%timeit
engines = planes.groupby(planes['engine'])[['engine']].value_counts().index
engine_counts = planes.groupby(planes['engine'])[['engine']].value_counts().values
task3_my = pd.DataFrame([engine_counts.T, engines]).T
task3_my.columns = ['COUNT(*)', 'engine']
task3_my['COUNT(*)'] = task3_my['COUNT(*)'].astype("int64")

2.68 ms ± 28.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%%timeit
task3_my2 = planes['engine'].value_counts().reset_index()
task3_my2.columns = ['engine', 'COUNT(*)']
task3_my2 = task3_my2.astype({'COUNT(*)': 'int64'})
task3_my2 = task3_my2.loc[:, ['COUNT(*)', 'engine']]

1.36 ms ± 17.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Unsurprisingly the inefficient method performed the slowest, particularly as it required multiple transposing of the data to wrangle the data into the correct format. However surprisingly, the SQL query was marginally faster than the more efficient pandas solution. However, it is likely that the additional steps required to massage the data to match the SQL output format are adding additional time. So it is possible that the pandas method is still faster, however the difference is likely to the negligible. 

## Task 4: "SELECT COUNT / GROUP BY" on multiple columns.

The next SQL query is similar to the previous, but this time it is performed on multiple columns. Specifically, this code reads the "planes" table and counts the number of rows grouped by engine and type. It returns a DataFrame that is sorted alphabetically by engine and then type. 

In [19]:
task4_sql=pd.read_sql_query("""SELECT COUNT(*), engine, type FROM planes
                               GROUP BY engine, type""", conn)
task4_sql

,COUNT(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine
5,2,Turbo-prop,Fixed wing multi engine
6,5,Turbo-shaft,Rotorcraft


### **PANDAS Equivalent of "SELECT COUNT / GROUP BY" on multiple columns.**

Similarly to the single column, this solution can be achieved in two ways. This time however, I will not manually assign the indices and values to seperate variables. This is actually not neccessary, and simply using the groupby() method along with reset_index() method will create the desired DataFrame object.

In [20]:
task4_my = planes.groupby(['engine', 'type'])[['engine', 'type']].value_counts().reset_index()
task4_my.columns = ['engine', 'type', 'COUNT(*)']
task4_my = task4_my[['COUNT(*)','engine', 'type']]
try:
    pd.testing.assert_frame_equal(task4_sql, task4_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


From here the code works just as it did earlier, but this time there are two columns instead of one. Also, the column names and orders need to be adjusted to match the SQL output and pass the assert test. This code is much simpler and cleaner than the previous version of it in the previous task and should perform better on testing.

The second solution is identical to the Task 3 solution, but this time the DataFrame is filtered by two columns instead of one.

In [21]:
task4_sql.sort_values(by="COUNT(*)", ascending=False, inplace=True)
task4_sql.reset_index(drop=True, inplace=True)

task4_my2 = planes[['engine', 'type']].value_counts().reset_index()
task4_my2.columns = ['engine', 'type','COUNT(*)']
task4_my2 = task4_my2[['COUNT(*)', 'engine', 'type']]
try:
    pd.testing.assert_frame_equal(task4_sql, task4_my2)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


As before, the row ordering needs to be adjusted on the SQL output to pass the assertion test. This solution will automatically sort the rows by the Count column in a descending order.

### **Task 4 - Performance test**

The results of the performance test are below.

In [22]:
%%timeit
pd.read_sql_query("""SELECT COUNT(*), engine, type FROM planes
                    GROUP BY engine, type""", conn)

1.93 ms ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
%%timeit
task4_my = planes.groupby(['engine', 'type'])[['engine', 'type']].value_counts().reset_index()
task4_my.columns = ['engine', 'type', 'COUNT(*)']
task4_my = task4_my[['COUNT(*)','engine', 'type']]

2.59 ms ± 37.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%%timeit
task4_my2 = planes[['engine', 'type']].value_counts().reset_index()
task4_my2.columns = ['engine', 'type','COUNT(*)']
task4_my2 = task4_my2[['COUNT(*)', 'engine', 'type']]

2.09 ms ± 93.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As before, it appears the that groupby method is considerably slower than applying value_counts directly on the subsection of the DataFrame. However, the SQL query was the fastest performer on this problem. It may still be that the additional steps to match the SQL output are slowing down the test on the pandas solutions, however the gap is starting to increase so it may be that the SQL query is faster for this type of problem. Again, it appears that as the problems become more complicated, SQL will start to perform better.

## Task 5: "SELECT MIN, MEAN, MAX OF  multiple GROUPED columns.

The next SQL query reads the "planes" table and performs minimum, maximum, and mean aggregations of the year based on the combined engine type and manufacturer.

In [25]:
task5_sql=pd.read_sql_query("""SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer
                               FROM planes 
                               GROUP BY engine, manufacturer""", conn)
task5_sql.head(5)

,MIN(year),AVG(year),MAX(year),engine,manufacturer
0,1975.0,1975.0,1975.0,4 Cycle,CESSNA
1,NaN,NaN,NaN,4 Cycle,JOHN G HESS
2,NaN,NaN,NaN,Reciprocating,AMERICAN AIRCRAFT INC
3,2007.0,2007.0,2007.0,Reciprocating,AVIAT AIRCRAFT INC
4,NaN,NaN,NaN,Reciprocating,BARKER JACK L


### **PANDAS Equivalent of "SELECT MIN, MEAN, MAX OF  multiple GROUPED columns.**

Once again, I will provide two solutions; an inefficient one and a more efficient one.

In [26]:
plane_mins = planes.groupby(['engine', 'manufacturer'])['year'].min().reset_index()
plane_means = planes.groupby(['engine', 'manufacturer'])['year'].mean().reset_index()
plane_maxs = planes.groupby(['engine', 'manufacturer'])['year'].max().reset_index()

result = pd.merge(plane_mins, plane_means, how='left', on=['engine', 'manufacturer'])
result = pd.merge(result, plane_maxs, how='left', on=['engine', 'manufacturer'])

result.rename(columns={'year_x': 'MIN(year)', 
                       'year_y': 'AVG(year)',
                       'year': 'MAX(year)'}, inplace=True)

task5_my = result[['MIN(year)','AVG(year)','MAX(year)','engine','manufacturer']]

try:
    pd.testing.assert_frame_equal(task5_sql, task5_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


The above solution individually performs the min, max and mean aggregations on the year column after grouping by the engine and manufacturer and saves these DataFrames as their own variables. Then to produce the final output, these three DataFrames are merged one by one into a single DataFrame (note: the pd.merge() method can only merge two DataFrames at a time). Then the columns are renamed and reorganized to match the SQL output and pass the assertion test.

While this works, there is a better way to solve the problem.

In [27]:
results = planes.groupby(['engine', 'manufacturer']).agg({'year': ['min', 'mean', 'max']}).reset_index()
results.columns = ['engine', 'manufacturer', 'MIN(year)', 'AVG(year)', 'MAX(year)']
task5_my2 = results[['MIN(year)','AVG(year)','MAX(year)','engine','manufacturer']]

try:
    pd.testing.assert_frame_equal(task5_sql, task5_my2)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


The process of combining the aggregated dataframes can be simplified by applying the agg() method to the output of the groupby method. The agg() method is a useful method as it can be passed a string, list or dictionary that specifies the desired column or aggregation function. In the above case, the dictionary specifies the column as the key and a list of desired aggregation functions as the value. This approach is ideal if different aggregation functions are to be applied to different columns. The same outcome above can can achieved by passing just the list of the aggregation functions to the agg() method, provided the column is specified as a filter. 

For example this code would also work:
```
planes.groupby(['engine', 'manufacturer'])['year'].agg(['min', 'mean', 'max']).reset_index()
```

Once this is done, as before, the columns are renamed and reorganized such that they match the SQL output.

### **Task 5 - Performance test**

The results of the performance test are below.

In [28]:
%%timeit
task5_my = pd.read_sql_query("""SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer
                               FROM planes 
                               GROUP BY engine, manufacturer""", conn)

3.1 ms ± 50.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%%timeit
plane_mins = planes.groupby(['engine', 'manufacturer'])['year'].min().reset_index()
plane_means = planes.groupby(['engine', 'manufacturer'])['year'].mean().reset_index()
plane_maxs = planes.groupby(['engine', 'manufacturer'])['year'].max().reset_index()

result = pd.merge(plane_mins, plane_means, how='left', on=['engine', 'manufacturer'])
result = pd.merge(result, plane_maxs, how='left', on=['engine', 'manufacturer'])

result.rename(columns={'year_x': 'MIN(year)', 
                       'year_y': 'AVG(year)',
                       'year': 'MAX(year)'}, inplace=True)

task5_my = result[['MIN(year)','AVG(year)','MAX(year)','engine','manufacturer']]

6.29 ms ± 94.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%%timeit
results = planes.groupby(['engine', 'manufacturer']).agg({'year': ['min', 'mean', 'max']}).reset_index()
results.columns = ['engine', 'manufacturer', 'MIN(year)', 'AVG(year)', 'MAX(year)']
task5_my2 = results[['MIN(year)','AVG(year)','MAX(year)','engine','manufacturer']]

2.47 ms ± 36.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The inefficient pandas group by methods were unsurprisingly the slowest performer on testing. However, this time the combination of groupby and agg performed signficantly better than the SQL queries. It appears that perhaps pandas may be better for performing aggregation functions than SQL.

## Task 6: "FILTER OUT NULL VALUES" on single columns

The next SQL query reads the planes table and simply filters out all the rows in the speed column that are empty.

In [31]:
task6_sql=pd.read_sql_query("""SELECT * FROM planes WHERE speed IS NOT NULL""", conn)
task6_sql.head(5)

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,424,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,427,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
2,821,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
3,893,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
4,1027,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating


### **PANDAS Equivalent of "FILTER OUT NULL VALUES" on single columns**

This can easily be replicated using Pandas by putting a filter on the DataFrame.

In [32]:
task6_my = planes[planes['speed'].notna()].reset_index()

try:
    pd.testing.assert_frame_equal(task6_sql, task6_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


The notna() method can be applied to the speed column to return a series of boolean values. This series can then be applied to the DataFrame to return only the rows within the speed column that contain values.

### **Task 6 - Performance test**

The results of the performance test are below.

In [33]:
%%timeit
task6_sql=pd.read_sql_query("""SELECT * FROM planes WHERE speed IS NOT NULL""", conn)

944 µs ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [34]:
%%timeit
task6_my = planes[planes['speed'].notna()].reset_index()

282 µs ± 4.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


THe results of the test show that the notna() method is atleast three times faster than the SQL query. Also, it is interesting to note that there was no additional wrangling required to match the SQL output format, so it suggests that the pandas methods may be even faster if they are not required to produce outputs in a specific format.

## Task 7: "SELECT" single column while "FILTERING" by multiple conditions

The next SQL query reads in the planes table, filters the rows by the column seat where the rows are between 150 and 210 and then returns only the tailnum column.

In [35]:
task7_sql=pd.read_sql_query("""SELECT tailnum FROM planes 
                               WHERE seats BETWEEN 150 AND 210 AND year >= 2011""", conn)
task7_sql.head(5)

,tailnum
0,N150UW
1,N151UW
2,N152UW
3,N153UW
4,N154UW


### **PANDAS Equivalent of "SELECT" single column while "FILTERING" by multiple conditions**

This can be achieved by chaining together the required seat column conditions and applying that as a filter to the DataFrame. 

In [36]:
task7_my = planes[(planes['seats']>=150)&
              (planes['seats']<=210)&
              (planes['year']>=2011)][['tailnum']].reset_index(drop=True)

try:
    pd.testing.assert_frame_equal(task7_sql, task7_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


The above code filters the entire DataFrame first by all the seat column conditions. Then the 'tailnum' column is selected and the index is reset. This is enough to pass the assertion test. 

### **Task 7 - Performance test**

The results of the performance test are below.

In [37]:
%%timeit
task7_sql=pd.read_sql_query("""SELECT tailnum FROM planes 
                               WHERE seats BETWEEN 150 AND 210 AND year >= 2011""", conn)

601 µs ± 3.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [38]:
%%timeit
task7_my = planes[(planes['seats']>=150)&
              (planes['seats']<=210)&
              (planes['year']>=2011)][['tailnum']].reset_index(drop=True)

624 µs ± 7.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The results show that for this task, the SQL query was the faster performer, however the difference between SQL and pandas in this case is negligible.

## Task 8: "SELECT" multiple columns while "FILTERING" on multiple conditions including a column subset.

The next SQL query is a little more complicated then the previous task. As before the planes table is read, but this time the "tailnum", "manufacturer", and "seats" columns are selected. Of these columns the rows where the manufacturer is one of "BOEING", "AIRBUS", or "EMBRAER", and the seats are greater than 390 are returned.

In [39]:
task8_sql=pd.read_sql_query("""SELECT tailnum, manufacturer, seats FROM planes
                               WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats > 390""",
                                 conn)
task8_sql.head(5)

,tailnum,manufacturer,seats
0,N206UA,BOEING,400
1,N228UA,BOEING,400
2,N272AT,BOEING,400
3,N57016,BOEING,400
4,N670US,BOEING,450


### **PANDAS Equivalent of "SELECT" multiple columns while "FILTERING" on multiple conditions including a column subset.**

Although the task is a little more complicated on the surface, it can be solved using the same pattern as the previous solution while taking advantage of the isin() method.

In [40]:
task8_my= planes[(planes['manufacturer'].isin(["BOEING", "AIRBUS", "EMBRAER"]))&
                 (planes['seats'] > 390)][['tailnum',
                                           'manufacturer', 
                                          'seats']].reset_index(drop=True)

try:
    pd.testing.assert_frame_equal(task8_sql, task8_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


First Dataframe is filtered by the neccessary conditions, which in this case is seats > 390 and the manufacturer is equal to either Boeing, Airbus or Embraer. The manufacturer condition can be used by passing these terms as a list into the isin() method. Next this output is filtered down to the "tailnum", "manufacturer", and "seats" columns. To pass the assertion test, the indices must be reset.

### **Task 8 - Performance test**

The results of the performance test are below.

In [41]:
%%timeit
task8_sql=pd.read_sql_query("""SELECT tailnum, manufacturer, seats FROM planes
                               WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats > 390""",
                                 conn)

895 µs ± 6.99 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [42]:
%%timeit
task8_my= planes[(planes['manufacturer'].isin(["BOEING", "AIRBUS", "EMBRAER"]))&
                 (planes['seats'] > 390)][['tailnum',
                                           'manufacturer', 
                                          'seats']].reset_index(drop=True)

647 µs ± 5.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The results show that the SQL query is approximetly ~30 - 40% slower than the pandas methods for this problem.

## Task 9: "SELECT UNIQUE" values on multiple columsn while "FILTERING and ORDERING" by single column: PART 1

The next SQL query is the first of two very similar queries that illustrate the importance of selecting the correct column to order by.

The query reads the planes table and selects the unique combinations of year and seats where the year column is greater than or equal to 2012. The output is then organised by year in an ascending order and seats in a descending order. In this case, the year is sorted first and then the seats are sorted.

In [43]:
task9_sql=pd.read_sql_query("""SELECT DISTINCT year, seats FROM planes
                               WHERE year >= 2012 ORDER BY year ASC, seats DESC""",
                                conn)
task9_sql.head(5)

,year,seats
0,2012.0,379
1,2012.0,377
2,2012.0,260
3,2012.0,222
4,2012.0,200


### **PANDAS Equivalent of "SELECT UNIQUE" values on multiple columsn while "FILTERING and ORDERING" by single column: PART 1**

This solution will utilize a combination of two methods that have been used when solving some of the prior tasks.

In [44]:
task9_my = planes[planes['year']>=2012][['year',
                            'seats']].drop_duplicates().sort_values(by=['year',
                                                                        'seats'],
                                                                    ascending=[True, 
                                                                            False]).reset_index(drop=True)

try:
    pd.testing.assert_frame_equal(task9_sql, task9_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


This code first filters the entire DataFrame by the condition where year is greater than r equl to 2012. Then the seat and year columns are selected and the drop_duplicates() method is applied to extract the unique combinations. Then the sort_values() method is called, however this time, to ensure that multi-column sorting is performed correctly, a list of the columns is passed in as a 'by' parameter. In addition, to ensure the sorting order for both columns is performed correctly, another boolean list is passed into the ascending parameter. 

Finally to pass the assertion test, the indices are reset.

### **Task 9 - Performance test**

The results of the performance test are below.

In [45]:
%%timeit
task9_sql=pd.read_sql_query("""SELECT DISTINCT year, seats FROM planes
                               WHERE year >= 2012 ORDER BY year ASC, seats DESC""",
                                conn)

594 µs ± 4.28 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [46]:
%%timeit
task9_my = planes[planes['year']>=2012][['year',
                            'seats']].drop_duplicates().sort_values(by=['year',
                                                                        'seats'],
                                                                    ascending=[True, 
                                                                            False]).reset_index(drop=True)

1.28 ms ± 43.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Interestingly, this test revealed that the SQL query was faster than the pandas methods by more than double. This is the first result where the disparity was this far in favour of SQL over pandas. Both methods required sorting, so the difference can be purely attributed to the underlying system.

## Task 10: "SELECT UNIQUE" values on multiple columns while "FILTERING and ORDERING" by single column: PART 2

In part 2, the query is essentially the same as part 1, except this time the columns are ordered by seats in descending order and then year in ascending order.

In [47]:
task10_sql=pd.read_sql_query("""SELECT DISTINCT year, seats FROM planes
                                WHERE year >= 2012 ORDER BY seats DESC, year ASC""",
                                  conn)
task10_sql.head(5)

,year,seats
0,2012.0,379
1,2013.0,379
2,2012.0,377
3,2013.0,377
4,2012.0,260


### **PANDAS Equivalent of "SELECT UNIQUE" values on multiple columns while "FILTERING and ORDERING" by single column: PART 2**

The exact same code can be recycled from the previous task with the neccessary amendments.

In [48]:
task10_my = planes[planes['year']>=2012][['year',
                            'seats']].drop_duplicates().sort_values(by=['seats',
                                                                        'year'],
                                                                    ascending=[False, 
                                                                            True]).reset_index(drop=True)

try:
    pd.testing.assert_frame_equal(task10_sql, task10_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


The only change from part one is the order of the list that is passed into the sort_values() method. This illustrates the importance of understanding the order in which the columns are to be sorted.

**NOTE: NO PERFORMANCE TEST WAS PERFORMED AS THIS PROCESS IS INDENTICAL TO THE PREVIOUS TASK**

## Task 11: "SELECT COUNT / GROUP BY" on single column while "FILTERING" by single condition

The next SQL query is an extension of the Count and Group by query from Task 3 but this time a condition is applied first. The planes table is read and a filter is applied where only the rows where the seat column is greater than 200 is selected. Then the grouping and counting is performed on the manufacturer column.

In [49]:
task11_sql=pd.read_sql_query("""SELECT manufacturer, COUNT(*) FROM planes 
                                WHERE seats > 200 GROUP BY manufacturer""", conn)
task11_sql

,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


### **PANDAS Equivalent of "SELECT COUNT / GROUP BY" on single column while "FILTERING" by single condition**

The pattern on how to replicate these queries using pandas only should start to be apparent, as the methods used have already been used to solve previous tasks. The pattern is to filter the DataFrame by the specified condtion, then select the desired columns and perform the desired aggregration or counting function.

In [50]:
task11_my = planes[planes['seats'] > 200].groupby("manufacturer")[['manufacturer']].value_counts().reset_index()
task11_my.columns = ["manufacturer", "COUNT(*)"]

try:
    pd.testing.assert_frame_equal(task11_sql, task11_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


First, to obtain the same sorting method that the SQL output naturally produces, the groupby() method is explicitly stated. This solution is able to produce the result in less lines, but the code is slightly more verbose and difficult to read.

On the other hand, the next segment will provide the same output.

In [51]:
task11_my2 = planes[planes['seats'] > 200]['manufacturer'].value_counts().reset_index()
task11_my2.columns = ["manufacturer", "COUNT(*)"]
task11_my2.sort_values(by='manufacturer', inplace=True)
task11_my2.reset_index(drop=True, inplace=True)

try:
    pd.testing.assert_frame_equal(task11_sql, task11_my2)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


This code does not use the groupby() method, and instead directly selects the desired column and applies the count after filtering by the seat condition. However, the sorting will be on Count by default. So to pass the assertion test, the rows must be sorted by the manufacturer column.

### **Task 11 - Performance test**

The results of the performance test are below.

In [52]:
%%timeit
task11_sql=pd.read_sql_query("""SELECT manufacturer, COUNT(*) FROM planes 
                                WHERE seats > 200 GROUP BY manufacturer""", conn)

611 µs ± 6.83 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [53]:
%%timeit
task11_my = planes[planes['seats'] > 200].groupby("manufacturer")[['manufacturer']].value_counts().reset_index()
task11_my.columns = ["manufacturer", "COUNT(*)"]

1.57 ms ± 60.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [54]:
%%timeit
task11_my2 = planes[planes['seats'] > 200]['manufacturer'].value_counts().reset_index()
task11_my2.columns = ["manufacturer", "COUNT(*)"]
task11_my2.sort_values(by='manufacturer', inplace=True)
task11_my2.reset_index(drop=True, inplace=True)

768 µs ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The results show that the groupby() method was again the slower performer of the pandas solutions. It is becoming apparent that if the task can be performed without using groupby(), then groupby() should be avoided. In this case, the SQL query was the best performer.

## Task 12: "SELECT COUNT / GROUP BY" on single column while "FILTERING" by single condition based on COUNT

This next SQL query provides an interesting spin on the previous task. 

The planes table is read, then the grouping and counting is performed on the manufacturer column. Then the filter by count is applied.

In [55]:
task12_sql=pd.read_sql_query("""SELECT manufacturer, COUNT(*) FROM planes 
                                GROUP BY manufacturer HAVING COUNT(*) > 10""", conn)
task12_sql

,manufacturer,COUNT(*)
0,AIRBUS,336
1,AIRBUS INDUSTRIE,400
2,BOEING,1630
3,BOMBARDIER INC,368
4,EMBRAER,299
5,MCDONNELL DOUGLAS,120
6,MCDONNELL DOUGLAS AIRCRAFT CO,103
7,MCDONNELL DOUGLAS CORPORATION,14


### **PANDAS Equivalent of "SELECT COUNT / GROUP BY" on single column while "FILTERING" by single condition based on COUNT**  

This time, the usual pattern that we have become accustomed to cannot be used as the filter must be applied after the grouping and count is performed.

In [56]:
mask = planes['manufacturer'].value_counts() > 10
task12_my = planes['manufacturer'].value_counts()[mask].reset_index()
task12_my.columns = ["manufacturer", "COUNT(*)"]
task12_my.sort_values(by='manufacturer', inplace=True)
task12_my.reset_index(drop=True, inplace=True)

try:
    pd.testing.assert_frame_equal(task12_sql, task12_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


To solve this problem, a masking variable can be created by applying the value_counts to the manufacturer column where the counts are greater than 10. Then this mask can be applied ontop of the same line of code, where reseting the index will create the desired DataFrame.

Then to pass the assertion test the columns are renamed and reordered to match the SQL output.

### **Task 12 - Performance test**

The results of the performance test are below.

In [57]:
%%timeit
task12_sql=pd.read_sql_query("""SELECT manufacturer, COUNT(*) FROM planes 
                                GROUP BY manufacturer HAVING COUNT(*) > 10""", conn)

1.56 ms ± 77.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [58]:
%%timeit
mask = planes['manufacturer'].value_counts() > 10
task12_my = planes['manufacturer'].value_counts()[mask].reset_index()
task12_my.columns = ["manufacturer", "COUNT(*)"]
task12_my.sort_values(by='manufacturer', inplace=True)
task12_my.reset_index(drop=True, inplace=True)

1.13 ms ± 39.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In this case, the pandas masking method was quicker than the SQL query. It is interesting to note that in the earlier tasks SQL outperformed pandas when filtering by a condition, but the mask approach was not used in those tasks.

## Task 13: "SELECT COUNT / GROUP BY" on single column while "FILTERING" by COUNT and other condition

The next SQL query extends upon the previous on by adding an extra condition to the count. The planes table is read and then the grouping and counting is performed on the manufacturer column. Then the filter by countand seats > 200 is applied.

In [59]:
task13_sql=pd.read_sql_query("""SELECT manufacturer, COUNT(*) FROM planes
                                WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10""",
                                conn)
task13_sql

,manufacturer,COUNT(*)
0,AIRBUS,66
1,BOEING,225


### **PANDAS Equivalent of "SELECT COUNT / GROUP BY" on single column while "FILTERING" by COUNT and other condition**

Once the previous method is understood, it is possible to expand the previous code to solve the problem.



In [60]:
mask = planes[planes['seats'] > 200]['manufacturer'].value_counts() > 10
task13_my = planes[planes['seats'] > 200]['manufacturer'].value_counts()[mask].reset_index()
task13_my.columns = ["manufacturer", "COUNT(*)"]
task13_my.sort_values(by='manufacturer', inplace=True)
task13_my.reset_index(drop=True, inplace=True)

try:
    pd.testing.assert_frame_equal(task13_sql, task13_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


The same masking procedure is followed, but this time the DataFrame is filtered by the other condition (seats > 200) prior to filtering by the value_counts.

This boolean filter is then applied to the same line of code. It is important that the syntax matches otherwise the boolean masking will not work properly. 

As usual, to pass the assertion test, the columns must be renamed and reordered.

### **Task 13 - Performance test**

The results of the performance test are below.

In [61]:
%%timeit
task13_sql=pd.read_sql_query("""SELECT manufacturer, COUNT(*) FROM planes
                                WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10""",
                                conn)

685 µs ± 21.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [62]:
%%timeit
mask = planes[planes['seats'] > 200]['manufacturer'].value_counts() > 10
task13_my = planes[planes['seats'] > 200]['manufacturer'].value_counts()[mask].reset_index()
task13_my.columns = ["manufacturer", "COUNT(*)"]
task13_my.sort_values(by='manufacturer', inplace=True)
task13_my.reset_index(drop=True, inplace=True)

1.44 ms ± 31.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In this case, the SQL query was faster than the pandas method even with masking. The difference however was that the mask itself required additional filtering which has likely slowed pandas down.

## Task 14: "SELECT COUNT / GROUP BY" on single column while "ORDERING"

The next SQL query changes things up slightly by changing the name of the returned column and performing a different sorting order. Also, the results are limited to just 10 rows.

The planes table is read and then the grouping and counting is performed on the manufacturer column. The count column is renamed 'howmany' and then the ordering and limiter is specified.

In [63]:
task14_sql=pd.read_sql_query("""SELECT manufacturer, COUNT(*) AS howmany
                                FROM planes
                                GROUP BY manufacturer
                                ORDER BY howmany DESC LIMIT 10""", conn)
task14_sql

,manufacturer,howmany
0,BOEING,1630
1,AIRBUS INDUSTRIE,400
2,BOMBARDIER INC,368
3,AIRBUS,336
4,EMBRAER,299
5,MCDONNELL DOUGLAS,120
6,MCDONNELL DOUGLAS AIRCRAFT CO,103
7,MCDONNELL DOUGLAS CORPORATION,14
8,CESSNA,9
9,CANADAIR,9


### **PANDAS Equivalent of "SELECT COUNT / GROUP BY" on single column while "ORDERING"** 

Up until now, because of the way SQL orders by default, it has been neccessary to call sort_values() to match. This time, the SQL will be ordered in such as way that is congruent with how Pandas orders the output.

In [64]:
task14_my = planes[['manufacturer']].value_counts().reset_index()[:10]
task14_my.columns = ["manufacturer", "howmany"]
try:
    pd.testing.assert_frame_equal(task14_sql, task14_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


The code is relatively similar to before, however this time to ensure that the limiter is applied, a split is applied to the DataFrame such that only the first 10 rows are returned.

### **Task 14 - Performance test**

The results of the performance test are below.

In [65]:
%%timeit
task14_sql=pd.read_sql_query("""SELECT manufacturer, COUNT(*) AS howmany
                                FROM planes
                                GROUP BY manufacturer
                                ORDER BY howmany DESC LIMIT 10""", conn)

1.62 ms ± 41.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [66]:
%%timeit
task14_my = planes[['manufacturer']].value_counts().reset_index()[:10]
task14_my.columns = ["manufacturer", "howmany"]

1.53 ms ± 52.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The results show that pandas performed better than SQL on this type of query, however the difference was not big. In an earlier similar task, SQL was slightly faster than pandas and it was hypothesized that pandas would be faster if there was no additional requirement to massage the DataFrame, and this result confirms that hypothesis.

## Task 15: "JOINING" columns from two dataframes

The next SQL query will be the first of a series where multiple tables are joined together.

This query reads data from two tables, "flights" and "planes". Then it selects all columns from the "flights" table, as well as the "year", "speed", and "seats" columns from the "planes" table while renaming the columns from the "planes" table. The resulting DataFrame is created by performing a left join of the two tables on the "tailnum" column.

A left join is a type of SQL join operation that returns all the rows from the left table (i.e., the table specified first in the join clause) and only the matching rows from the right table. If there are no matching rows in the right table, then the result will contain null values for the columns in the right table. 

In this case, "flights" will be the left table and "planes" will be the right table.

In [67]:
task15_sql=pd.read_sql_query("""SELECT
                                    flights.*,
                                    planes.year AS plane_year,
                                    planes.speed AS plane_speed,
                                    planes.seats AS plane_seats 
                                FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum""",
                                conn)
task15_sql.head(5)

,index,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,...,origin,dest,air_time,distance,hour,minute,time_hour,plane_year,plane_speed,plane_seats
0,0,2013,1,1,517.0,515,2.0,830.0,819,11.0,...,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,1999.0,NaN,149.0
1,1,2013,1,1,533.0,529,4.0,850.0,830,20.0,...,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,1998.0,NaN,149.0
2,2,2013,1,1,542.0,540,2.0,923.0,850,33.0,...,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00,1990.0,NaN,178.0
3,3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,...,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00,2012.0,NaN,200.0
4,4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,...,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,1991.0,NaN,178.0


### **PANDAS Equivalent of "JOINING" columns from two dataframes**

This task is easy to replicate using the pandas.merge() method; a method that was used a little earlier in one of the inefficient solutions to a prior task.

In [68]:
planes_renamed = planes[['tailnum', 
                         'year', 
                         'speed', 
                         'seats']].rename(columns={'year': 'plane_year', 
                                                   'speed': 'plane_speed', 
                                                   'seats': 'plane_seats'})
task15_my = pd.merge(flights, planes_renamed, how='left', on='tailnum').reset_index()

try:
    pd.testing.assert_frame_equal(task15_sql, task15_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


First, the neccesary renaming is performed prior to joining the two Dataframes. The columns that are selected from planes, 'tailnum', 'year', 'speed' and 'seats' are stored as a variable while the rename() method is called to make the neccessary name changes. The 'tailnum' column is selected in this instance because it also exists in the flights DataFrame.

Then, pandas.merge() passing the flights DataFrame as the left and the planes DataFrame as the right. 'Left' is passed as an argument to the 'how' parameter which performs the left join, while the 'on' parameter receives the name of the column that is common to both tables.

The indices are reset which results in the assertion test passing.

### **Task 15 - Performance test**

The results of the performance test are below.

In [69]:
%%timeit
task15_sql=pd.read_sql_query("""SELECT
                                    flights.*,
                                    planes.year AS plane_year,
                                    planes.speed AS plane_speed,
                                    planes.seats AS plane_seats 
                                FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum""",
                                conn)

2.85 s ± 92.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [70]:
%%timeit
planes_renamed = planes[['tailnum', 
                         'year', 
                         'speed', 
                         'seats']].rename(columns={'year': 'plane_year', 
                                                   'speed': 'plane_speed', 
                                                   'seats': 'plane_seats'})
task15_my = pd.merge(flights, planes_renamed, how='left', on='tailnum').reset_index()

143 ms ± 5.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


When performing a simple left join, pandas is monumentally faster than SQL. 

## Task 16: "JOINING" columns from three dataframes with "INNER JOIN"

The next SQL query increases in complexity by joining three tables instead of two.

The query reads data from three tables called "flights", "planes", and "airlines". It selects all columns from the "planes" and "airlines" tables, and performs an inner join of "planes" and "airlines" on "cartail". The table "cartail" represents the unique combinations of carrier and tailnum from the "flights" table which is renamed as "cartail".

An inner join is a type of SQL join that returns only the rows from both tables where there is a match based on the joining column. If there is no match, the row is not included in the result set. So, in other words, if the result of the join is a null value, it is not included.

In [71]:
task16_sql=pd.read_sql_query("""SELECT planes.*, airlines.* FROM
                                (SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
                                INNER JOIN planes ON cartail.tailnum=planes.tailnum
                                INNER JOIN airlines ON cartail.carrier=airlines.carrier""",
                                conn)
task16_sql


,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine,index,carrier,name
0,0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan,5,EV,ExpressJet Airlines Inc.
1,1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,12,US,US Airways Inc.
2,2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,12,US,US Airways Inc.
3,3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,12,US,US Airways Inc.
4,4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan,5,EV,ExpressJet Airlines Inc.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3334,3317,N997AT,2002.0,Fixed wing multi engine,BOEING,717-200,2,100,NaN,Turbo-fan,7,FL,AirTran Airways Corporation
3335,3318,N997DL,1992.0,Fixed wing multi engine,MCDONNELL DOUGLAS AIRCRAFT CO,MD-88,2,142,NaN,Turbo-fan,4,DL,Delta Air Lines Inc.
3336,3319,N998AT,2002.0,Fixed wing multi engine,BOEING,717-200,2,100,NaN,Turbo-fan,7,FL,AirTran Airways Corporation
3337,3320,N998DL,1992.0,Fixed wing multi engine,MCDONNELL DOUGLAS CORPORATION,MD-88,2,142,NaN,Turbo-jet,4,DL,Delta Air Lines Inc.


### **PANDAS Equivalent of  "JOINING" columns from three dataframes with "INNER JOIN"**

This query is difficult to replicate in pandas without making an adjustment to the SQL output. I was not able to decipher how SQL performs the joins, or rather which order the joins happen. While I was able to replicate the basic elements of the joins, the SQL output retains the original indices of the rows throughout the joins and it was these specific columns that I could not replicate in order to pass the assertion test.

So, I elected to drop the 'index' column from the SQL output and proceed with my solution.

In [72]:
task16_sql.drop(['index'], axis=1, inplace=True)

cartail = flights[['carrier', 'tailnum']].drop_duplicates()
task16_my = pd.merge(planes, cartail, how='inner', on='tailnum')
task16_my= pd.merge(task16_my, airlines, how='inner', on='carrier')
task16_my.sort_values(by=['tailnum', 'carrier'], inplace=True)
task16_my.reset_index(drop=True, inplace=True)


try:
    pd.testing.assert_frame_equal(task16_sql, task16_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)


Test passed: Outputs are Equal


First the 'cartail' DataFrame is created by finding the unique combinations using the drop_duplicates() method. Then two pandas.merge() joins are performed to join the DataFrames together while specifying 'inner' as the join type.

Finally, the values are sorted by tailnum and carrier and the indices are dropped in order to pass the assertion test. 

### **Task 16 - Performance test**

The results of the performance test are below.

In [73]:
%%timeit
task16_sql=pd.read_sql_query("""SELECT planes.*, airlines.* FROM
                                (SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
                                INNER JOIN planes ON cartail.tailnum=planes.tailnum
                                INNER JOIN airlines ON cartail.carrier=airlines.carrier""",
                                conn)
task16_sql.drop(['index'], axis=1, inplace=True)

240 ms ± 4.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
%%timeit
cartail = flights[['carrier', 'tailnum']].drop_duplicates()
task16_my = pd.merge(planes, cartail, how='inner', on='tailnum')
task16_my= pd.merge(task16_my, airlines, how='inner', on='carrier')
task16_my.sort_values(by=['tailnum', 'carrier'], inplace=True)
task16_my.reset_index(drop=True, inplace=True)

43.1 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


As with the previous task, pandas outperforms SQL on this particular task. However, it is important to note that the dispartiy in perfromance is much narrower. It seems that as the joins become more complicated, SQL may begin to outperform pandas.

## Task 17: "JOINING" "FILTERED/AVERAGED" columns with another dataframe 

In this final task, perhaps the most difficult join of all the tasks is performed by this next SQL query.

The query involves joining two tables together, however the join is not performed on the original tables, but instead two subqueries are performed on the "flights" and "weather" tables and the output of these subqueries are joined together.

In the first instance on the "weather" table, first the table is filtered by those rows where the origin column is equal to 'EWR'. Then for those rows, the year month and day columns are grouped together and an aggregation mean function is applied to the temperature and humidity columns. The resulting year, month, day, average temp and average humidity columns are selected where average temp andaverage humidity are renamed 'atemp' and 'ahumid' respectively. This subquery is given the name 'weather2' and this will be the left table in the join.

Then in the next subquery on the "flights" table, all rows where the origin column is equal to 'EWR' are selected. This subquery is given the name 'flights2' which shall be the right table in the join.

Once the left join is performed, all columns that were in 'flights2' along with the 'atemp' and 'ahumid' columns from  'weather2' are returned.

In [75]:
task17_sql=pd.read_sql_query("""SELECT
                                    flights2.*,
                                    atemp,
                                    ahumid
                                FROM ( 
                                    SELECT * FROM flights WHERE origin='EWR'
                                ) AS flights2
                                LEFT JOIN ( 
                                    SELECT 
                                        year, month, day,
                                        AVG(temp) AS atemp,
                                        AVG(humid) AS ahumid
                                    FROM weather
                                    WHERE origin='EWR'
                                    GROUP BY year, month, day
                                ) AS weather2
                                ON flights2.year=weather2.year
                                AND flights2.month=weather2.month
                                AND flights2.day=weather2.day""", conn)
task17_sql

,index,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,...,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,atemp,ahumid
0,0,2013,1,1,517.0,515,2.0,830.0,819,11.0,...,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,38.4800,58.386087
1,5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,...,N39463,EWR,ORD,150.0,719,5,58,2013-01-01 05:00:00,38.4800,58.386087
2,6,2013,1,1,555.0,600,-5.0,913.0,854,19.0,...,N516JB,EWR,FLL,158.0,1065,6,0,2013-01-01 06:00:00,38.4800,58.386087
3,13,2013,1,1,558.0,600,-2.0,923.0,937,-14.0,...,N53441,EWR,SFO,361.0,2565,6,0,2013-01-01 06:00:00,38.4800,58.386087
4,16,2013,1,1,559.0,600,-1.0,854.0,902,-8.0,...,N76515,EWR,LAS,337.0,2227,6,0,2013-01-01 06:00:00,38.4800,58.386087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120830,336752,2013,9,30,2142.0,2129,13.0,2250.0,2239,11.0,...,N12957,EWR,PWM,47.0,284,21,29,2013-09-30 21:00:00,62.9075,69.806250
120831,336755,2013,9,30,2149.0,2156,-7.0,2245.0,2308,-23.0,...,N813UA,EWR,BOS,37.0,200,21,56,2013-09-30 21:00:00,62.9075,69.806250
120832,336756,2013,9,30,2150.0,2159,-9.0,2250.0,2306,-16.0,...,N10575,EWR,MHT,39.0,209,21,59,2013-09-30 21:00:00,62.9075,69.806250
120833,336760,2013,9,30,2211.0,2059,72.0,2339.0,2242,57.0,...,N12145,EWR,STL,120.0,872,20,59,2013-09-30 20:00:00,62.9075,69.806250


### **PANDAS Equivalent of "JOINING" "FILTERED/AVERAGED" columns with another dataframe**

As before, while I could replicate the general content of this query, I was unable to replicate this query using pandas such that it would pass the assertion test without dropping the indices from the SQL output.

In [76]:
task17_sql.drop(['index'], axis=1, inplace=True)

flights2 = flights[flights['origin'] == 'EWR']
averages = weather[weather['origin']=='EWR'].groupby(['year', 'month', 'day'])[['temp','humid']].mean().reset_index()
averages.rename(columns={'temp': 'atemp', 'humid':'ahumid'}, inplace=True)
weather2 = weather[weather['origin']=='EWR'].merge(averages, how='left', on=['year', 'month', 'day'])[['year', 'month', 'day', 'atemp', 'ahumid']]

task17_my = pd.merge(flights2, weather2, how='left', on=['year', 'month', 'day']).drop_duplicates().reset_index(drop=True)

try:
    pd.testing.assert_frame_equal(task17_sql, task17_my)
    print("Test passed: Outputs are Equal")
except AssertionError as e:
    print("Test failed:", e)

Test passed: Outputs are Equal


The first step is to create the flights2 DataFrame by filtering the flights DataFrame based on the condition that 'origin' = 'EWR'.

Next, before the weather2 DataFrame could be constructed, the aggregated means of the humidity and temperature columns needed to be computed. This was done by creating a variable called 'averages' which would be used to combine with the weather DataFrame to create the weather2 DataFrame. To compute the averages, the weather DataFrame was first filtered by the condition 'origin' = 'EWR'. Then the groupby() method is applied which groups the DataFrame by year, month and day. Then the temp and humid columns are selected and the mean().method is applied. Resetting the index returns the output into a DataFrame format. The columns are renamed 'atemp' and 'ahumid' to meet the requirements.

Then, this averages variable is merged with the weather DataFrame which is also filtered down such that only the rows where the origin column is equal to 'EWR' are selected. This completes the weather2 DataFrame creation step.

Then finally, these two DataFrames are merged together using the pandas.merge() method. However, because there were multiple flights that occurred on the same year/month/day, the result is a number of duplicated rows in this output DataFrame. So the drop_duplicates() method is called to eliminate these duplicates. Then the indices are reset to pass the assertion test.

### **Task 17 - Performance test**

The results of the performance test are below.

In [77]:
%%timeit
task17_sql=pd.read_sql_query("""SELECT
                                    flights2.*,
                                    atemp,
                                    ahumid
                                FROM ( 
                                    SELECT * FROM flights WHERE origin='EWR'
                                ) AS flights2
                                LEFT JOIN ( 
                                    SELECT 
                                        year, month, day,
                                        AVG(temp) AS atemp,
                                        AVG(humid) AS ahumid
                                    FROM weather
                                    WHERE origin='EWR'
                                    GROUP BY year, month, day
                                ) AS weather2
                                ON flights2.year=weather2.year
                                AND flights2.month=weather2.month
                                AND flights2.day=weather2.day""", conn)

task17_sql.drop(['index'], axis=1, inplace=True)

919 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
%%timeit
flights2 = flights[flights['origin'] == 'EWR']
averages = weather[weather['origin']=='EWR'].groupby(['year', 'month', 'day'])[['temp','humid']].mean().reset_index()
averages.rename(columns={'temp': 'atemp', 'humid':'ahumid'}, inplace=True)
weather2 = weather[weather['origin']=='EWR'].merge(averages, how='left', on=['year', 'month', 'day'])[['year', 'month', 'day', 'atemp', 'ahumid']]

task17_my = pd.merge(flights2, weather2, how='left', on=['year', 'month', 'day']).drop_duplicates().reset_index(drop=True)

2.06 s ± 50.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The results show that SQL now outperforms pandas by a decent margin when performing this type of join. As with the results in previous tests, when pandas is required to perform filtering then the performance starts to decrease.

## Conclusion

In summary, This exercise began with a naive theory and hypothesis regarding pandas and SQL which was partially supported. The results demonstrate that the exact same outputs can be acheived using SQL queries or pandas methods and that depending on the task at hand one approach may outperform the other. For instance, while pandas may be faster on the whole, it seems that as the outputs become more complicated with more filtering, then SQL may be better. The implications of these findings boil down to choosing the right data infrastructures for the tasks to be performed. There are advantages to using SQL to store data in databases as opposed to maintaining data in csv spreadsheets that may outweigh the potential peformance advatanges that pandas may have over SQL in terms of returning the output. This exercise was a novice approach to creating a taxonomy of output types and determing whether pandas or SQL performed better. A future study may wish to create a more robust taxonomy of the task types with clear definitions for the outputs. Then proper hypothesis tests for performance can be generated for each task type and the results be reported.